In [90]:
import pandas as pd
from pymongo import MongoClient

df = pd.read_csv('../data/Covid_Data_Clean.csv')

client = MongoClient('mongodb://localhost:27017/')
db = client['CovidDB']
collection = db['patients']

# Prvo cemo pre unosa obrisati sve stare podatke da ne bi duplirali
collection.delete_many({})


DeleteResult({'n': 387382, 'ok': 1.0}, acknowledged=True)

In [91]:
data_dict = df.to_dict(orient='records')
collection.insert_many(data_dict)

print(f"Uspešno uvezeno {collection.count_documents({})} dokumenata u MongoDB.")

Uspešno uvezeno 387382 dokumenata u MongoDB.


In [92]:
# Funkcija za pomoć pri ispisu rezultata
def report_query(title, pipeline):
    print(f"\n--- {title} ---")
    results = list(collection.aggregate(pipeline))
    if results:
        report_df = pd.DataFrame(results)
        print(report_df.to_string(index=False))
    else:
        print("Nema rezultata za ovaj upit.")
    return results

# ISTRAŽIVAČKO PITANJE 1 - Distribucije ishoda virusa prema starosti i polu

In [93]:
# Procenat smrtnosti po starosnim grupama
q1 = [
    {"$group": {
        "_id": "$AGE_GROUP",
        "total": {"$sum": 1},
        "deaths": {"$sum": {"$cond": [{"$eq": ["$DEATH", 1]}, 1, 0]}}
    }},
    {"$project": {
        "death_percentage": {"$multiply": [{"$divide": ["$deaths", "$total"]}, 100]}
    }},
    {"$sort": {"_id": 1}}
]

In [94]:
# Odnos polova kod hospitalizovanih pacijenata
q2 = [
    {
    "$group": {
            "_id": "$SEX", 
            "stopa_smrtnosti": {"$avg": "$DEATH"} 
        }
    }
]

In [95]:
# Prosecna starost preminulih po polu
q3 = [
    {"$match": {"DEATH": 1}},
    {"$group": {"_id": "$SEX", "avg_age": {"$avg": "$AGE"}}}
]

# ISTRAŽIVAČKO PITANJE 2 - Faktori hospitalizacije

In [96]:
# Procenat hospitalizacije za pacijente sa dijabetesom
q4 = [
    {"$group": {
        "_id": "$DIABETES",
        "hosp_rate": {"$avg": "$HOSPITALIZED"}
    }}
]

# Uticaj gojaznosti na hospitalizaciju
q5 = [
    {"$group": {
        "_id": "$OBESITY",
        "hosp_count": {"$sum": "$HOSPITALIZED"},
        "total": {"$sum": 1}
    }}
]

# Top 3 medicinske ustanove po broju hospitalizovanih
q6 = [
    {"$match": {"HOSPITALIZED": 1}},
    {"$group": {"_id": "$MEDICAL_UNIT", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 3}
]

# ISTRAŽIVAČKO PITANJE 3 - Komorbiditeti i kritična stanja

In [97]:
# Verovatnoća da pacijent bude na ICU u odnosu na broj hroničnih bolesti
q7 = [
    {"$group": {
        "_id": "$COMORBIDITY_GROUP",
        "icu_prob": {"$avg": {"$cond": [{"$eq": ["$ICU", 1]}, 1, 0]}}
    }},
    {"$sort": {"_id": 1}}
]

# Broj pacijenata sa 3+ bolesti koji su završili na respiratoru
q8 = [
    {
        "$match": {
            # Tražimo tačan string ovde jer je kolona tekstualna
            "COMORBIDITY_GROUP": "3+", 
            "INTUBED": 1.0 
        }
    },
    {
        "$count": "high_risk_intubed"
    }
]

# Najčešća pojedinačna bolest kod pacijenata na intenzivnoj nezi
# Najčešća stanja kod pacijenata na intenzivnoj nezi
q9 = [
    {"$match": {"ICU": 1}}, 
    {"$group": {
        "_id": None,
        "pneumonija": {"$sum": {"$cond": [{"$eq": ["$PNEUMONIA", 1]}, 1, 0]}},
        "dijabetes": {"$sum": {"$cond": [{"$eq": ["$DIABETES", 1]}, 1, 0]}},
        "hipertenzija": {"$sum": {"$cond": [{"$eq": ["$HIPERTENSION", 1]}, 1, 0]}},
        "gojaznost": {"$sum": {"$cond": [{"$eq": ["$OBESITY", 1]}, 1, 0]}}
    }}
]

# ISTRAŽIVAČKO PITANJE 4: Pneumonija, starost i smrtnost

In [98]:
# Smrtnost kod pacijenata sa pneumonijom u odnosu na starosnu grupu
q10 = [
    {"$match": {"PNEUMONIA": 1}},
    {"$group": {
        "_id": "$AGE_GROUP",
        "death_rate": {"$avg": "$DEATH"}
    }},
    {"$sort": {"_id": 1}}
]

# Verovatnoća pneumonije kod pušača vs nepušača
q11 = [
    {"$group": {
        "_id": "$TOBACCO",
        "pneumonia_rate": {"$avg": {"$cond": [{"$eq": ["$PNEUMONIA", 1]}, 1, 0]}}
    }}
]

# Prosečna starost pacijenata koji imaju i pneumoniju i dijabetes, a preživeli su
q12 = [
    {"$match": {"PNEUMONIA": 1, "DIABETES": 1, "DEATH": 0}},
    {"$group": {"_id": None, "avg_age": {"$avg": "$AGE"}}}
]

# DODATNI ANALITIČKI UPITI

In [99]:
# Koliko imamo preživelih vs preminulih
q13 = [
    {"$group": {"_id": "$DEATH", "count": {"$sum": 1}}}
]

# Identifikacija "High Risk" pacijenata (Stariji od 60 sa Pneumonijom)
q14 = [
    {"$match": {"AGE": {"$gt": 60}, "PNEUMONIA": 1}},
    {"$group": {"_id": "$DEATH", "count": {"$sum": 1}}}
]

# Udeo kardiovaskularnih bolesnika u ukupnoj smrtnosti
q15 = [
    {"$match": {"CARDIOVASCULAR": 1}},
    {"$group": {"_id": "$DEATH", "count": {"$sum": 1}}}
]


In [100]:
all_queries = [
    (q1, "Smrtnost po starosti"), (q2, "Smrtnost po polu"), (q3, "Starost hospitalizovanih"),
    (q4, "Gojaznost vs Hosp"), (q5, "Dijabetes vs Hosp"), (q6, "Top units"),
    (q7, "ICU po komorbiditetima"), (q8, "Kritični 3+"), (q9, "Najčešće bolesti kod ICU pacijenata"),
    (q10, "Pneumonija kod starih"), (q11, "Pneumonija kod pušača"), (q12, "Prosečna starost preživelih sa pneumonijom i dijabetesom"),
    (q13, "Balans klasa"), (q14, "Visokorizični pacijenti (pneumonija i 60)"), (q15, "Udeo kardiovaskularnih bolesnika u smrtnim slučajevima")
]

for query, title in all_queries:
    report_query(title, query)


--- Smrtnost po starosti ---
  _id  death_percentage
 0-19          1.568190
20-39          2.188098
40-59         12.109798
60-79         37.619547
  80+         52.975888

--- Smrtnost po polu ---
 _id  stopa_smrtnosti
   1         0.101074
   0         0.164077

--- Starost hospitalizovanih ---
 _id   avg_age
   0 60.924098
   1 62.737681

--- Gojaznost vs Hosp ---
 _id  hosp_rate
 1.0   0.556353
 0.0   0.227596

--- Dijabetes vs Hosp ---
 _id  hosp_count  total
 1.0       25085  71867
 0.0       83188 315515

--- Top units ---
 _id  count
   4  50213
  12  36530
   6   8401

--- ICU po komorbiditetima ---
_id  icu_prob
  0  0.002861
  1  0.032433
  2  0.059790
 3+  0.082700

--- Kritični 3+ ---
 high_risk_intubed
              7629

--- Najčešće bolesti kod ICU pacijenata ---
 _id  pneumonija  dijabetes  hipertenzija  gojaznost
None        8881       3325          3574       2860

--- Pneumonija kod starih ---
  _id  death_rate
 0-19    0.163218
20-39    0.190056
40-59    0.403020

In [101]:
cursor = collection.aggregate([
    {
        "$group": {
            "_id": "$COMORBIDITY_GROUP", 
            "icu_rate": {"$avg": "$ICU"}
        }
    },
    {
        "$sort": {"_id": 1}
    }
])

# Pretvaranje u DataFrame za vizuelizaciju
df_icu_analiza = pd.DataFrame(list(cursor))

df_icu_analiza.rename(columns={'_id': 'Broj_Bolesti'}, inplace=True)

print(df_icu_analiza)

  Broj_Bolesti  icu_rate
0            0  0.002861
1            1  0.032433
2            2  0.059790
3           3+  0.082700


# Tumačenje rezultata upita

q1 - Eksponencijalni rast smrtnosti, tj. vidimo da smrtnost drastično skače sa godinama. 
U grupi 0-19 svega 1.57% smrtnosti, dok je kod pacijenata starijih od 80 godina ona 52.98%.

q2 - Vidimo da muškarci imaju veću verovatnoću smrtnog ishoda (oko 16%) u odnosu na žene (oko 10%)

q3 - Nema velike razlike u godinama između onih koji su bili kod kuće i onih u bolnici (oko 61-62 godine), što sugeriše da su se u bolnice primali pacijenti slične starosne dobi, ali verovatno različite kliničke slike.

q4 - Vidimo da je šansa za hospitalizaciju kod gojaznih osoba 55.6%, dok je kod onih koji nisu gojazni ta šansa više nego duplo manja,tj. 22.7%.

q5 - Od ukupno 71,867 dijabetičara, njih 25,085 je hospitalizovano, što potvrđuje dijabetes kao jedan od primarnih okidača za bolničko lečenje

q6 - Medicinske jedinice 4 i 12 su primile ubedljivo najviše pacijenata, što ukazuje na to da su se u njima i najviše trošili resursi

q7 - Vidimo da verovatnoća završetka na intenzivnoj nezi raste sa svakom novom bolešću, tj. što je veći broj bolesti veći je i rizik od kritičnog stanja

q8 - Vidimo da imamo 7,629 visokorizičnih pacijenata  (onih sa 3+ bolesti koji su završili na respiratoru)

q9 - Vidimo da pacijenti koji su na intenzivnoj nezi imaju najčešće pneumoniju, dok su dijabetes i hipertenzija prisutni kod velikog dela te grupe

q10 - Kod pacijenata starijih od 80 godina koji imaju pneumoniju, šansa za smrtnost je 72.4%.

q11 - Pušači češće imaju pneumoniju (24,1%) u odnosu na nepušače (21,5%). Ovo može ukazivati da pacijenti zbog pušenja mogu razviti težu kliničku sliku

q12 - Prosečna starost preživelih sa pneumonijom i dijabetesom je oko 57.5 godina

q13 - Prisutan je veliki disbalans, imamo 335.206 preživelih i 52.176 preminulih, što znači da ćemo morati da obratimo pažnju na ovo kada budemo radili u Spark MLu

q14 - Među pacijentima starijim od 60 godina sa pneumonijom, broj preminulih (21.435) je skoro duplo veći od broja onih koji su preživeli (11.901)

q15 - Od malo više od 8000 kardiovaskularnih bolesnika, njih 2.759 je preminulo